<a href="https://colab.research.google.com/github/zhhvvv/continuous-integration/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# # DataPath = 'CUB_200_2011/CUB_200_2011'
# DataPath = '../CUB_200_2011/CUB_200_2011'
# MaskPath = '../segmentations'

# classNumber = 200
# LogPath = '/result/'
# SaveModel = './checkpoint/'
#!tar -xzvf /content/drive/MyDrive/Colab_Notebooks/CUB_200_2011.tgz

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!tar -xzvf /content/drive/MyDrive/Colab_Notebooks/CUB_200_2011.tgz

In [ ]:
!pip install imageio
!pip install einops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 532 kB/s 


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import tqdm
from torch.utils.data import DataLoader
import imageio
from einops import rearrange

from PIL import Image

from imgaug import augmenters as iaa
from torchvision import transforms

from torchvision import models
from torch.nn import init

import torchvision
import torch.nn.functional as F

from torch.utils.data.dataset import Dataset
import cv2 as cv
import os
import random

import pandas as pd

In [ ]:
DataPath = '/content/CUB_200_2011'
# DataPath = '../CUB_200_2011/CUB_200_2011'
# MaskPath = '../segmentations'
# DataPath = '../CUB_200_2011/CUB_200_2011'

classNumber = 200
LogPath = '/content/drive/MyDrive/record/'
SaveModel = '/content/drive/MyDrive/checkpoint/'

In [ ]:
class CUB_200_2011(Dataset):
    def __init__(self, dataSetName, shape, train=True, path=DataPath, aug=False, T=None):
        super(CUB_200_2011, self).__init__()
        self.dataSetName = dataSetName
        self._FOR_TRAIN = train
        self._shape = shape
        self.path = path
        self.data_path = []
        self.data_label = []
        self.aug = aug
        self._Init()
        if T is None:
            self.T = transforms.Compose([
                iaa.SomeOf((0, 2), [
                    iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)),
                    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05 * 255), per_channel=0.5),
                    iaa.Superpixels(p_replace=(0, 1.0), n_segments=(20, 200)),
                    iaa.PiecewiseAffine(scale=(0.01, 0.05)),
                    iaa.ChannelShuffle(0.35)]).augment_image
                # transforms.ToTensor()
            ])
        else:
            self.T = T

    def _Init(self):
        target = '1' if self._FOR_TRAIN else '0'
        data_index = []
        with open(self.path + '/train_test_split.txt', 'r') as F:
            for i in F.readlines():
                a = i.split()
                if a[1] == target:
                    data_index.append(a[0])

        with open(self.path + '/images.txt', 'r') as F:
            for i in F.readlines():
                a = i.split()
                if a[0] in data_index:
                    _ = a[1].split('.')
                    label = int(_[0])
                    self.data_label.append(label)
                    self.data_path.append(a[1])

    def __getitem__(self, item):

        img = cv.imread(self.path + '/images/' + self.data_path[item])

        if len(img.shape) == 1:
          img = cv.cvtColor(img, cv.CV_GRAY2RGB)

        img = cv.resize(img, (self._shape, self._shape))
        label = int(self.data_path[item].split("/")[-2].split(".")[0])

        if self.aug:
            img = self.T(img)
            # cv.imshow("ok", img)
            # cv.waitKey(0 )

        data = torch.Tensor(img)
        data = rearrange(data, "w h c -> c w h")
        return data, torch.LongTensor([label - 1])

    def __len__(self):
        return len(self.data_label)

In [ ]:
class evalMatrix:
    def __init__(self, clses, device, Matrix=None):
        if Matrix is None:
            print(clses, device)
            # a = torch.from_numpy(np.zeros((clses, clses)))
            # a = a.to(device)
            self.Matrix = torch.from_numpy(np.zeros((clses, clses))).to(device)
            print('ok')
        else:
            self.Matrix = Matrix
        self.device = device
        self.clses = clses
        self.acc = -1
        self.recall = -1
        self.precision = -1
        self.f1 = -1
        self.kappa = -1
        # print(self.Matrix)

    def record(self, out, y):
        pred = torch.argmax(out, dim=1)
        for i in range(len(pred)):
            self.Matrix[y[i].item()][max(0, pred[i].item())] += 1

    def clear(self):
        self.Matrix = torch.from_numpy(np.zeros((self.clses, self.clses))).to(self.device)

    def analysis(self):
        all_number = torch.sum(self.Matrix)
        # print(all_number)

        # get acc
        acc = 0
        for i in range(self.clses):
            acc += self.Matrix[i][i]
        acc = acc / all_number
        self.acc = acc.item()
        # print(acc)

        # get recall
        recall = torch.zeros(self.clses).to(self.device)
        for i in range(self.clses):
            recall[i] = self.Matrix[i][i] / torch.sum(self.Matrix, dim=0)[i]
        self.recall = recall.sum().item() / self.clses

        # print(recall)
        precision = torch.zeros(self.clses).to(self.device)
        for i in range(self.clses):
            precision[i] = self.Matrix[i][i] / torch.sum(self.Matrix, dim=1)[i]
        self.precision = precision.sum().item() / self.clses

        # get f1
        f1 = torch.torch.zeros(self.clses).to(self.device)
        for i in range(self.clses):
            f1[i] = 2 * precision[i] * recall[i] / (precision[i] + recall[i])
        self.f1 = f1.sum().item() / self.clses
        # print(f1)

        # get kappa
        pe = 0
        for i in range(self.clses):
            pe += torch.sum(self.Matrix, dim=1)[i] * torch.sum(self.Matrix, dim=0)[i]
        pe = pe / (torch.sum(self.Matrix) ** 2)
        kappa = (acc - pe) / (1 - pe)
        self.kappa = kappa.item()
        # print(kappa)

        return self.acc, self.recall, self.precision, self.f1, self.kappa


In [ ]:
def weight_init_kaiming(m):
    class_names = m.__class__.__name__
    if class_names.find('Conv') != -1:
        init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
    elif class_names.find('Linear') != -1:
        init.kaiming_normal_(m.weight.data)
        
def box(img, mask):
    if len(img.shape) == 2:
        new_img = np.zeros((img.shape[0], img.shape[1], 3))
        for i in range(3):
            new_img[:, :, i] = img
        img = new_img
    if len(mask.shape) == 3:
        mask = mask[:, :, 0]

    maskimg = cv.bitwise_and(src1=img, src2=img, mask=mask)
    # print(maskimg.shape)
    sub_img = np.where(maskimg > 0)
    final = maskimg[min(sub_img[0]):max(sub_img[0]), min(sub_img[1]):max(sub_img[1]), :]
    return final


def set_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


class Log:
    def __init__(self, path, model_name, df_style=None):
        self.path = path
        self.model_name = model_name
        if df_style is None:
            self.df_style = pd.DataFrame(
                columns=['epoch', 'train_loss', 'train_acc', 'train_precision', 'train_recall', 'train_f1',
                         'train_kappa',
                         'val_loss', 'val_acc', 'val_precision', 'val_recall', 'val_f1', 'val_kappa',
                         'test_acc', 'test_precision', 'test_recall', 'test_f1', 'test_kappa'],
                index=[i for i in range(50)],
            )
        else:
            self.df_style = df_style

        # print(self.df_style)

    def load(self, path=None):
        if path is None:
            self.df_style = pd.read_csv(self.path, index_col=-1)

        else:
            self.df_style = pd.read_csv(path, index_col=-1)

        print('load succeed!')

    def print(self):
        print(self.df_style)

    def record(self, epoch, evaluator, loss, state, auto_write=False):
        r = list(evaluator.analysis())
        self.df_style.iloc[epoch]['epoch'] = epoch
        if state != 'test':
            self.df_style.iloc[epoch][state + '_loss'] = loss
        self.df_style.iloc[epoch][state + '_acc'] = r[0]
        self.df_style.iloc[epoch][state + '_precision'] = r[1]
        self.df_style.iloc[epoch][state + '_recall'] = r[2]
        self.df_style.iloc[epoch][state + '_f1'] = r[3]
        self.df_style.iloc[epoch][state + '_kappa'] = r[4]
        if auto_write:
            self.write()
        else:
            print(
                'The record has been updated, but not yet written to the file because the argument \'auto_write\' is False')

    def write(self):
        self.df_style.to_csv(self.path, index=False)
        print('log succeed!')


def count_parameters(model):
    # 统计一个模型的参数
    temp = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f'\nThe model has {temp:,} trainable parameters')


def get_device():
    return 'cuda:0' if torch.cuda.is_available() else 'cpu'


def score(state, evaluater, epoch=-1, loss=-1.):
    acc, recall, pre, f1, kappa = evaluater.analysis()
    if state is 'test':
        print('|acc       : {}%'.format(acc * 100))
        print('|recall    : {}'.format(recall))
        print('|precision : {}'.format(pre))
        print('|f1        : {}'.format(f1))
        print('|kappa     : {}'.format(kappa))
    else:
        print('|epoch     : {}    '.format(epoch) + state + ' loss: {}'.format(loss))
        print('|acc       : {}%'.format(acc * 100))
        print('|recall    : {}'.format(recall))
        print('|precision : {}'.format(pre))
        print('|f1        : {}'.format(f1))
        print('|kappa     : {}'.format(kappa))


In [ ]:
def test(model, device, dataset, batch_size, test_evaluater=None, log=None, shuffle=True, num_workers=2):
    test_set = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
    pred = _test(model, test_set, device, test_evaluater)
    if log:
        log.record(0, test_evaluater, -1, 'test', auto_write=True)
    score('test', test_evaluater)
    return pred


def _test(model, input_loader, device, evaluater=None):
    evaluater.clear()
    model.eval()
    with torch.no_grad():
        for src, cls in tqdm.tqdm(input_loader, desc='test'):
            src = src.to(device)
            output = model(src).to(device)
            evaluater.record(output, cls)
            pred = torch.argmax(output, dim=1)
    return pred


In [ ]:
def train(model, train_data, val_data, epochs=3, batch_size=64, learning_rate=0.01, device='cpu', loss_func=None,
          optimizer=None, train_evaluator: evalMatrix = None, val_evaluator: evalMatrix = None, log=None, num_workers=2,
          save_model='best_model.pt'):
    """
    model: nn model
    data: each item is a tuple (image, label)
    epochs: int
    batch_size: int
    learning_rate: float
    """

    # Prepare parameter
    if not loss_func:
        loss_func = nn.CrossEntropyLoss()

    # Split data into batches
    train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True,
                                  num_workers=num_workers)
    val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=True, drop_last=True,
                                num_workers=num_workers)

    # Optimizer
    if optimizer is None:
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

    model.to(device)

    # Start training
    best_acc = -1
    for epoch in range(epochs):
        print("=" * 10, "Epoch", epoch, "=" * 10)
        total_loss = 0
        train_evaluator.clear()
        val_evaluator.clear()
        model.train()
        for data, cls in tqdm.tqdm(train_dataloader):
            # for data, cls in dataloader:
            # zero the gradients
            data = data.to(device)
            cls = cls.to(device)
            optimizer.zero_grad()
            # forward
            outputs = model(data)
            cls = torch.flatten(cls)
            loss = loss_func(outputs, cls)
            if train_evaluator is not None:
                train_evaluator.record(outputs, cls)
            # backward
            loss.backward()
            # optimize
            optimizer.step()
            # statistics
            total_loss += loss.data.item()

        print('avg_loss:', total_loss / (len(train_dataloader)))

        if train_evaluator is not None and log is not None:
            log.record(epoch=epoch, evaluator=train_evaluator, loss=float(total_loss) / len(train_dataloader),
                       state='train',
                       auto_write=True)

        _validation(model, criterion=loss_func, loader=val_dataloader, evaluater=val_evaluator, device=device)

        if val_dataloader is not None and log is not None:
            log.record(epoch=epoch, evaluator=val_evaluator, loss=0, state='test', auto_write=True)
            if best_acc < val_evaluator.acc:
                best_acc = val_evaluator.acc
                torch.save(model, SaveModel + save_model)

        scheduler.step(total_loss)


def _validation(model, criterion, loader, device, evaluater=None):
    evaluater.clear()
    model.eval()
    epoch_loss_ = 0
    with torch.no_grad():
        for src, cls in tqdm.tqdm(loader, desc='test '):
            src = src.to(device)
            cls = cls.to(device)
            output = model(src)
            cls = torch.flatten(cls)
            evaluater.record(output, cls)
            loss = criterion(output, cls)
            epoch_loss_ += loss.item()
    return epoch_loss_ / len(loader)


In [ ]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        
        self.Flatten = nn.Flatten()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=24, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(24)
        self.conv2 = nn.Conv2d(in_channels=24, out_channels=24, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(24)
        self.pool = nn.MaxPool2d(2,2)
        self.conv4 = nn.Conv2d(in_channels=24, out_channels=48, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(48)
        self.conv5 = nn.Conv2d(in_channels=48, out_channels=48, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(48)
        self.fc1 = nn.Linear(602112, 200)
        
        

    def forward(self, input):
        output = F.relu(self.bn1(self.conv1(input)))      
        output = F.relu(self.bn2(self.conv2(output)))     
        output = self.pool(output)                        
        output = F.relu(self.bn4(self.conv4(output)))     
        output = F.relu(self.bn5(self.conv5(output)))     
        #output = output.view(-1, 24*10*10)
        output = self.Flatten(output)
        output = self.fc1(output)
        
        return output

In [ ]:
# ! pip install transformers

# from transformers import ViTFeatureExtractor, ViTModel
# import torch
# feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
# pre_model = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

# device = get_device()
# pre_model.to(device)
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc=torch.nn.Linear(768,200)
    def forward(self,input):

        image_list=[im for im in input.cpu()]
        inputs = feature_extractor(image_list, return_tensors="pt").to(device)
        
        with torch.no_grad():
          out = pre_model(**inputs).last_hidden_state[:,0,:]
        
        out=self.fc(out)
        #out=out.softmax(dim=1)
        # out=out.softmax(dim=1)
        return out
#model_nn=Model()

class ResNet(nn.Module):
    def __init__(self, pre_trained=True, n_class=200, model_choice=50):
        super(ResNet, self).__init__()
        self.n_class = n_class
        self.base_model = self._model_choice(pre_trained, model_choice)
        self.base_model.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.base_model.fc = nn.Linear(512 * 4, n_class)
        self.base_model.fc.apply(weight_init_kaiming)

    def forward(self, x):
        N = x.size(0)
        assert x.size() == (N, 3, 192, 192)
        x = self.base_model(x)
        assert x.size() == (N, self.n_class)
        return x

    def _model_choice(self, pre_trained, model_choice):
        if model_choice == 50:
            return models.resnet50(pretrained=pre_trained)
        elif model_choice == 101:
            return models.resnet101(pretrained=pre_trained)
        elif model_choice == 152:
            return models.resnet152(pretrained=pre_trained)

In [ ]:
def main():
    os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
    # ------------------------------------------------------------
    # |                  hyper-parameter pool                    |
    # |                                                          |

    epochs = 100
    lr = 1e-3
    loss_func = nn.CrossEntropyLoss()


    # |                                                          |
    # |                  hyper-parameter pool                    |
    # ------------------------------------------------------------

    # train_data = R()

    #train_data = Raw_CUB_200_2011('cub', train=True)
    train_data = CUB_200_2011('cub', train=True, shape = 224, aug=True)
    test_data = CUB_200_2011('cub', train=False, shape = 224, aug=True)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #device = get_device()

    print(device)
    
    train_evaluator = evalMatrix(clses=classNumber, device=device)
    test_evaluator = evalMatrix(clses=classNumber, device=device)

    log = Log(LogPath + 'record.csv', model_name='cnn')

    # -----------------

    # vgg16 = VGG16(8, conv_paras, pool_paras, fc_paras)
    testmodel = Network()
    # testmodel=Model()
    #testmodel = ResNet(model_choice=50)
    count_parameters(testmodel)

    

    #train(testmodel, train_data, device = device, evaluator=train_evaluator, log=log)
    train(testmodel, train_data, test_data,
          epochs=2000,
          learning_rate=lr,
          device=device,
          loss_func=loss_func,
          num_workers=2,
          train_evaluator=train_evaluator,
          val_evaluator=test_evaluator,
          log=log)
    #test(model=testmodel, dataset=test_data, device=device, batch_size=64, test_evaluater=test_evaluator, log=log)


    # train(testmodel, train_data, device = device, evaluator=train_evaluator, log=log)
    # test(model=testmodel, dataset=test_data, device=device, batch_size=64, test_evaluater=test_evaluator, log=log)

    # -----------------


if __name__ == '__main__':
    main()

cuda:0
200 cuda:0
ok
200 cuda:0
ok

The model has 120,459,968 trainable parameters
========== Epoch 0 ==========


  0%|          | 0/93 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/imgaug/augmenters/segmentation.py:272: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.
  image, n_segments=n_segments_samples[i], compactness=10)
/usr/local/lib/python3.7/dist-packages/imgaug/augmenters/segmentation.py:272: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.
  image, n_segments=n_segments_samples[i], compactness=10)
  9%|▊         | 8/93 [00:18<03:31,  2.48s/it]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fc37d6f67a0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3

avg_loss: 224.0023211202314
log succeed!


test :   0%|          | 0/90 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/imgaug/augmenters/segmentation.py:272: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.
  image, n_segments=n_segments_samples[i], compactness=10)
/usr/local/lib/python3.7/dist-packages/imgaug/augmenters/segmentation.py:272: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.
  image, n_segments=n_segments_samples[i], compactness=10)
test :   4%|▍         | 4/90 [00:07<02:14,  1.57s/it]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fc37d6f67a0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__
    self._shutdown_workers()
  File "/usr/loc

log succeed!
========== Epoch 1 ==========


...

avg_loss: 1.2670420344157884
log succeed!


test :   0%|          | 0/90 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/imgaug/augmenters/segmentation.py:272: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.
  image, n_segments=n_segments_samples[i], compactness=10)
/usr/local/lib/python3.7/dist-packages/imgaug/augmenters/segmentation.py:272: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.
  image, n_segments=n_segments_samples[i], compactness=10)
test :   8%|▊         | 7/90 [00:17<02:52,  2.08s/it]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fc37d6f67a0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__
    self._shutdown_workers()
  File "/usr/loc

log succeed!
========== Epoch 11 ==========


  0%|          | 0/93 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/imgaug/augmenters/segmentation.py:272: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.
  image, n_segments=n_segments_samples[i], compactness=10)
/usr/local/lib/python3.7/dist-packages/imgaug/augmenters/segmentation.py:272: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.
  image, n_segments=n_segments_samples[i], compactness=10)
  5%|▌         | 5/93 [00:13<03:53,  2.65s/it]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fc37d6f67a0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3

KeyboardInterrupt: ignored

In [ ]:
!apt install psmisc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
psmisc is already the newest version (23.1-1ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.


In [ ]:
!sudo fuser /dev/nvidia*

/dev/nvidia0:           79m
/dev/nvidiactl:         79m
/dev/nvidia-uvm:        79m


In [ ]:
!kill -9 79

In [ ]:
# dataset1 = CUB(root='/content/CUB_200_2011', is_train=False, transform=transform_train,)
# print(len(dataset1))
# testloader=torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True, num_workers=0,
#                                               drop_last=True)
# print(len(testloader))

In [ ]:
# from torchvision.datasets import CIFAR10
# from torchvision.transforms import transforms
# from torch.utils.data import DataLoader

# Loading and normalizing the data.
# Define transformations for the training and test sets
# transformations = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
# ])

# CIFAR10 dataset consists of 50K training images. We define the batch size of 10 to load 5,000 batches of images.
# batch_size = 64
# number_of_labels = 200 

# Create an instance for training. 
# When we run this code for the first time, the CIFAR10 train dataset will be downloaded locally. 
#train_set =CIFAR10(root="./data",train=True,transform=transformations,download=True)

# Create a loader for the training set which will read the data within batch size and put into memory.
#train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
#print("The number of images in a training set is: ", len(train_loader)*batch_size)

# Create an instance for testing, note that train is set to False.
# When we run this code for the first time, the CIFAR10 test dataset will be downloaded locally. 
#test_set = CIFAR10(root="./data", train=False, transform=transformations, download=True)

# Create a loader for the test set which will read the data within batch size and put into memory. 
# Note that each shuffle is set to false for the test loader.
#test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0)
#print("The number of images in a test set is: ", len(test_loader)*batch_size)

#print("The number of batches per epoch is: ", len(train_loader))
# classes = (i for i in range(200))

In [ ]:
# import torch
# import torch.nn as nn
# import torchvision
# import torch.nn.functional as F

# # Define a convolution neural network
# class Network(nn.Module):
#     def __init__(self):
#         super(Network, self).__init__()
        
#         self.Flatten = nn.Flatten()

#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5, stride=1, padding=1)
#         self.bn1 = nn.BatchNorm2d(12)
#         self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=5, stride=1, padding=1)
#         self.bn2 = nn.BatchNorm2d(12)
#         self.pool = nn.MaxPool2d(2,2)
#         self.conv4 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=5, stride=1, padding=1)
#         self.bn4 = nn.BatchNorm2d(24)
#         self.conv5 = nn.Conv2d(in_channels=24, out_channels=24, kernel_size=5, stride=1, padding=1)
#         self.bn5 = nn.BatchNorm2d(24)
#         self.fc1 = nn.Linear(269664, 200)
        
        

#     def forward(self, input):
#         output = F.relu(self.bn1(self.conv1(input)))      
#         output = F.relu(self.bn2(self.conv2(output)))     
#         output = self.pool(output)                        
#         output = F.relu(self.bn4(self.conv4(output)))     
#         output = F.relu(self.bn5(self.conv5(output)))     
#         #output = output.view(-1, 24*10*10)
#         output = self.Flatten(output)
#         output = self.fc1(output)
        
#         return output

# # Instantiate a neural network model 
# model = Network()

In [ ]:
# from torch.optim import Adam
 
# # Define the loss function with Classification Cross-Entropy loss and an optimizer with Adam optimizer
# loss_fn = nn.CrossEntropyLoss()
# optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

In [ ]:
# from torch.autograd import Variable

# # Function to save the model
# def saveModel():
#     path = "./myFirstModel.pth"
#     torch.save(model.state_dict(), path)

# # Function to test the model with the test dataset and print the accuracy for the test images
# def testAccuracy():
    
#     model.eval()
#     accuracy = 0.0
#     total = 0.0
#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#     with torch.no_grad():
#         for data in testloader:
            
#             images, labels = data
#             images, labels = images.to(device), labels.to(device)
#             # run the model on the test set to predict labels
#             outputs = model(images)
#             # the label with the highest energy will be our prediction
#             _, predicted = torch.max(outputs.data, 1)
#             total += labels.size(0)
#             accuracy += (predicted == labels).sum().item()
    
#     # compute the accuracy over all test images
#     accuracy = (100 * accuracy / total)
#     return(accuracy)


# # Training function. We simply have to loop over our data iterator and feed the inputs to the network and optimize.
# def train(num_epochs):
    
#     best_accuracy = 0.0

#     # Define your execution device
#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#     #device = torch.device("cpu")
#     print("The model will be running on", device, "device")
#     # Convert model parameters and buffers to CPU or Cuda
#     model.to(device)

#     for epoch in range(num_epochs):  # loop over the dataset multiple times
#         running_loss = 0.0
        # running_acc = 0.0

        # for i, (images, labels) in enumerate(trainloader, 0):
        #     # images, labels = images.cuda(), labels.cuda()
        #     # get the inputs
        #     images = images.to(device)
        #     labels = labels.to(device)

        #     # zero the parameter gradients
        #     optimizer.zero_grad()
        #     # predict classes using images from the training set
        #     outputs = model(images)
        #     # compute the loss based on model output and real labels
        #     loss = loss_fn(outputs, labels)
        #     # backpropagate the loss
        #     loss.backward()
            # adjust parameters based on the calculated gradients
        #     optimizer.step()

        #     # Let's print statistics for every 1,000 images
        #     running_loss += loss.item()     # extract the loss value
        #     if i % 1000 == 999:    
        #         # print every 1000 (twice per epoch) 
        #         print('[%d, %5d] loss: %.3f' %
        #               (epoch + 1, i + 1, running_loss / 1000))
        #         # zero the loss
        #         running_loss = 0.0

        # # Compute and print the average accuracy fo this epoch when tested over all 10000 test images
        # accuracy = testAccuracy()
        # print('For epoch', epoch+1,'the test accuracy over the whole test set is %d %%' % (accuracy))
        
        # # we want to save the model if the accuracy is the best
        # if accuracy > best_accuracy:
        #     saveModel()
        #     best_accuracy = accuracy

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np

# # Function to show the images
# def imageshow(img):
#     img = img / 2 + 0.5     # unnormalize
#     npimg = img.numpy()
#     plt.imshow(np.transpose(npimg, (1, 2, 0)))
#     plt.show()


# # Function to test the model with a batch of images and show the labels predictions
# def testBatch():
#     # get batch of images from the test DataLoader  
#     images, labels = next(iter(testloader))

#     # show all images as one image grid
#     imageshow(torchvision.utils.make_grid(images))
   
#     # Show the real labels on the screen 
#     print('Real labels: ', ' '.join('%5s' % classes[labels[j]] 
#                                for j in range(batch_size)))
  
    # # Let's see what if the model identifiers the  labels of those example
    # outputs = model(images)
    
    # # We got the probability for every 10 labels. The highest (max) probability should be correct label
    # _, predicted = torch.max(outputs, 1)
    
    # # Let's show the predicted labels on the screen to compare with the real ones
    # print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] 
    #                           for j in range(batch_size)))

In [ ]:
# if __name__ == "__main__":
    
#     # Let's build our model
#     train(5)
#     print('Finished Training')

#     # Test which classes performed well
#     testAccuracy()
    # # Let's load the model we just created and test the accuracy per label
    # model = Network()
    # path = "myFirstModel.pth"
    # model.load_state_dict(torch.load(path))

    # # Test with batch of images

In [ ]:
# def count_parameters(model):
#     # 统计一个模型的参数
#     temp = sum(p.numel() for p in model.parameters() if p.requires_grad)
#     print(f'\nThe model has {temp:,} trainable parameters')
# count_parameters(model)